### Import Libraries and Load Data

In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os

# Create Spark Session
spark = SparkSession.builder \
    .appName("Project2 - Analytical Queries") \
    .config("spark.sql.warehouse.dir", os.path.abspath("../lakehouse")) \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✓ Spark Session created")
print(f"  Version: {spark.version}")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
✓ Spark Session created
  Version: 4.1.0


25/12/18 04:00:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Load All Layers and Dimensions

In [15]:
# Define paths
bronze_output = "../lakehouse/sales/bronze"
silver_output = "../lakehouse/sales/silver"

# Load Silver layer (cleaned fact table)
df_sales = spark.read.parquet(silver_output)

# Load Dimension tables
df_dim_customers = spark.read.csv("../data/customers.csv", header=True, inferSchema=True)
from pyspark.sql.window import Window
window_spec = Window.orderBy("customer_reference_id")
df_dim_customers = df_dim_customers.withColumn("customer_key", row_number().over(window_spec)) \
    .select(
        col("customer_key"),
        col("customer_reference_id"),
        col("first_name"),
        col("last_name"),
        col("email"),
        col("city"),
        col("age"),
        col("loyalty_tier")
    ).withColumn(
        "loyalty_member",
        when(col("loyalty_tier").isin("Gold", "Platinum"), "Yes").otherwise("No")
    )

df_dim_products = spark.read.csv("../data/products.csv", header=True, inferSchema=True)
window_spec = Window.orderBy("item_name")
df_dim_products = df_dim_products.withColumn("product_key", row_number().over(window_spec)) \
    .select(
        col("product_key"),
        col("item_name"),
        col("category"),
        col("brand"),
        col("material"),
        col("season"),
        col("gender_target"),
        col("base_price"),
        col("stock_quantity")
    )

# Generate Date Dimension
from datetime import datetime, timedelta
dates = []
start_date = datetime(2023, 1, 1)
for i in range(365):
    date = start_date + timedelta(days=i)
    dates.append({
        'date_key': i + 1,
        'purchase_date': date.strftime('%Y-%m-%d'),
        'full_date': date,
        'year': date.year,
        'month': date.month,
        'day': date.day,  # FIXED: Changed from day.day to date.day
        'quarter': (date.month - 1) // 3 + 1,
        'month_name': date.strftime('%B')
    })
df_dim_date = spark.createDataFrame(dates)

# Payment Dimension
payment_data = [
    {'payment_key': 1, 'payment_method': 'Credit Card'},
    {'payment_key': 2, 'payment_method': 'PayPal'},
    {'payment_key': 3, 'payment_method': 'Cash'},
    {'payment_key': 4, 'payment_method': 'Debit Card'}
]
df_dim_payment = spark.createDataFrame(payment_data)

print("="*60)
print("DATA LOADED FOR ANALYSIS")
print("="*60)
print(f"✓ Sales (Silver): {df_sales.count()} records")
print(f"✓ Customers: {df_dim_customers.count()} records")
print(f"✓ Products: {df_dim_products.count()} records")
print(f"✓ Dates: {df_dim_date.count()} records")
print(f"✓ Payment Methods: {df_dim_payment.count()} records")
print("="*60)

DATA LOADED FOR ANALYSIS
✓ Sales (Silver): 3400 records
✓ Customers: 166 records
✓ Products: 61 records
✓ Dates: 365 records
✓ Payment Methods: 4 records


### Query 1: Top 10 Customers by Total Spending

In [16]:
# Join sales with customers and aggregate
df_query1 = df_sales \
    .join(df_dim_customers, "customer_key") \
    .groupBy("customer_key", "first_name", "last_name", "email", "city", "loyalty_member") \
    .agg(
        count("customer_key").alias("num_purchases"),
        sum("purchase_amount").alias("total_spent"),
        avg("purchase_amount").alias("avg_purchase"),
        max("purchase_amount").alias("largest_purchase")
    ) \
    .orderBy(desc("total_spent")) \
    .limit(10)

print("="*80)
print("QUERY 1: TOP 10 CUSTOMERS BY TOTAL SPENDING")
print("="*80)
df_query1.show(10, truncate=False)

QUERY 1: TOP 10 CUSTOMERS BY TOTAL SPENDING
+------------+----------+---------+----------------------------+-------------+--------------+-------------+-----------+------------------+----------------+
|customer_key|first_name|last_name|email                       |city         |loyalty_member|num_purchases|total_spent|avg_purchase      |largest_purchase|
+------------+----------+---------+----------------------------+-------------+--------------+-------------+-----------+------------------+----------------+
|153         |Ethan     |Wright   |ethan.wright347@email.com   |Las Vegas    |No            |16           |9685.0     |745.0             |4932.0          |
|84          |Henry     |Allen    |henry.allen723@email.com    |San Francisco|Yes           |25           |9657.0     |536.5             |4872.0          |
|88          |Mason     |Mitchell |mason.mitchell485@email.com |Portland     |No            |24           |8745.0     |485.8333333333333 |4771.0          |
|119         |Emily 

25/12/18 04:02:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:02:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:02:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:02:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:02:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:02:20 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### Query 2: Product Performance by Category and Season

In [18]:
# Join sales with products and dates, filter non-null dates
df_query2 = df_sales \
    .filter(col("date_key").isNotNull()) \
    .join(df_dim_products, "product_key") \
    .join(df_dim_date, "date_key") \
    .groupBy("category", "season", "quarter") \
    .agg(
        count("customer_key").alias("num_sales"),
        sum("purchase_amount").alias("total_revenue"),
        avg("purchase_amount").alias("avg_sale_price"),
        count_distinct("customer_key").alias("unique_customers")
    ) \
    .orderBy(desc("total_revenue")) \
    .limit(15)

print("="*80)
print("QUERY 2: PRODUCT PERFORMANCE BY CATEGORY AND SEASON")
print("="*80)
df_query2.show(15, truncate=False)

QUERY 2: PRODUCT PERFORMANCE BY CATEGORY AND SEASON


25/12/18 04:05:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:05:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:05:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:05:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------------+-------+---------+-------------+------------------+----------------+
|category   |season       |quarter|num_sales|total_revenue|avg_sale_price    |unique_customers|
+-----------+-------------+-------+---------+-------------+------------------+----------------+
|Accessories|Fall/Winter  |2      |73       |14408.0      |228.6984126984127 |53              |
|Outerwear  |Spring/Summer|2      |63       |13345.0      |242.63636363636363|54              |
|Accessories|All Season   |1      |66       |13007.0      |240.87037037037038|56              |
|Bottoms    |Spring/Summer|3      |77       |11752.0      |186.53968253968253|68              |
|Tops       |Fall/Winter  |2      |50       |11168.0      |286.35897435897436|41              |
|Tops       |All Season   |1      |94       |10596.0      |137.6103896103896 |71              |
|Tops       |Spring/Summer|1      |52       |10381.0      |241.41860465116278|44              |
|Accessories|Fall/Winter  |3      |82   

### Query 3: Payment Method Analysis by Customer Loyalty

In [19]:
# Join sales with customers and payment methods
df_query3 = df_sales \
    .join(df_dim_customers, "customer_key") \
    .join(df_dim_payment, "payment_key") \
    .groupBy("loyalty_member", "payment_method") \
    .agg(
        count("customer_key").alias("num_transactions"),
        sum("purchase_amount").alias("total_amount"),
        avg("purchase_amount").alias("avg_transaction"),
        count_distinct("customer_key").alias("unique_customers")
    ) \
    .orderBy("loyalty_member", desc("total_amount"))

print("="*80)
print("QUERY 3: PAYMENT METHOD PREFERENCES BY LOYALTY STATUS")
print("="*80)
df_query3.show(truncate=False)

QUERY 3: PAYMENT METHOD PREFERENCES BY LOYALTY STATUS


25/12/18 04:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:06:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------+--------------+----------------+------------+------------------+----------------+
|loyalty_member|payment_method|num_transactions|total_amount|avg_transaction   |unique_customers|
+--------------+--------------+----------------+------------+------------------+----------------+
|No            |Credit Card   |817             |116559.0    |173.9686567164179 |75              |
|No            |Cash          |753             |78527.0     |129.79669421487603|75              |
|Yes           |Cash          |877             |121811.0    |172.2927864214993 |91              |
|Yes           |Credit Card   |953             |114055.0    |148.50911458333334|91              |
+--------------+--------------+----------------+------------+------------------+----------------+



### Query 4: Monthly Revenue Trends

In [20]:
# Time-series analysis of sales over months
df_query4 = df_sales \
    .filter(col("date_key").isNotNull()) \
    .join(df_dim_date, "date_key") \
    .groupBy("year", "month", "month_name") \
    .agg(
        count("customer_key").alias("num_transactions"),
        sum("purchase_amount").alias("monthly_revenue"),
        avg("purchase_amount").alias("avg_transaction_value"),
        count_distinct("customer_key").alias("active_customers")
    ) \
    .orderBy("year", "month")

print("="*80)
print("QUERY 4: MONTHLY REVENUE TRENDS")
print("="*80)
df_query4.show(12, truncate=False)

QUERY 4: MONTHLY REVENUE TRENDS
+----+-----+----------+----------------+---------------+---------------------+----------------+
|year|month|month_name|num_transactions|monthly_revenue|avg_transaction_value|active_customers|
+----+-----+----------+----------------+---------------+---------------------+----------------+
|2023|1    |January   |292             |29887.0        |126.10548523206751   |139             |
|2023|2    |February  |251             |33879.0        |171.1060606060606    |128             |
|2023|3    |March     |277             |39380.0        |169.74137931034483   |132             |
|2023|4    |April     |295             |40220.0        |171.14893617021278   |141             |
|2023|5    |May       |308             |45352.0        |181.408              |144             |
|2023|6    |June      |262             |35542.0        |170.05741626794259   |129             |
|2023|7    |July      |303             |33176.0        |134.8617886178862    |143             |
|2023|8 

### Query 5: Customer Lifetime Value by City and Loyalty

In [22]:
# Geographic and loyalty analysis
df_query5 = df_sales \
    .join(df_dim_customers, "customer_key") \
    .groupBy("city", "loyalty_member") \
    .agg(
        count_distinct("customer_key").alias("num_customers"),
        count("customer_key").alias("total_purchases"),
        sum("purchase_amount").alias("total_revenue"),
        avg("purchase_amount").alias("avg_purchase_value"),
        (sum("purchase_amount") / count_distinct("customer_key")).alias("revenue_per_customer")
    ) \
    .orderBy(desc("revenue_per_customer")) \
    .limit(20)

print("="*80)
print("QUERY 5: CUSTOMER LIFETIME VALUE BY CITY AND LOYALTY STATUS")
print("="*80)
df_query5.show(20, truncate=False)

QUERY 5: CUSTOMER LIFETIME VALUE BY CITY AND LOYALTY STATUS


25/12/18 04:07:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------+--------------+-------------+---------------+-------------+------------------+--------------------+
|city         |loyalty_member|num_customers|total_purchases|total_revenue|avg_purchase_value|revenue_per_customer|
+-------------+--------------+-------------+---------------+-------------+------------------+--------------------+
|Portland     |No            |4            |88             |22078.0      |319.9710144927536 |5519.5              |
|San Francisco|Yes           |3            |57             |15694.0      |348.75555555555553|5231.333333333333   |
|San Jose     |Yes           |2            |49             |8716.0       |207.52380952380952|4358.0              |
|San Antonio  |Yes           |4            |87             |17161.0      |241.70422535211267|4290.25             |
|Austin       |Yes           |1            |26             |4275.0       |251.47058823529412|4275.0              |
|Detroit      |No            |4            |94             |16208.0      |210.49

### Query 6: Brand Performance Analysis

In [23]:
# Analyze brand performance across different metrics
df_query6 = df_sales \
    .join(df_dim_products, "product_key") \
    .groupBy("brand", "category") \
    .agg(
        count("customer_key").alias("num_sales"),
        sum("purchase_amount").alias("brand_revenue"),
        avg("purchase_amount").alias("avg_price"),
        avg("review_rating").alias("avg_rating"),
        count_distinct("customer_key").alias("unique_customers")
    ) \
    .filter(col("num_sales") >= 10) \
    .orderBy(desc("brand_revenue")) \
    .limit(15)

print("="*80)
print("QUERY 6: TOP BRAND PERFORMANCE BY CATEGORY")
print("="*80)
df_query6.show(15, truncate=False)

QUERY 6: TOP BRAND PERFORMANCE BY CATEGORY


25/12/18 04:07:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------------------+-----------+---------+-------------+------------------+------------------+----------------+
|brand             |category   |num_sales|brand_revenue|avg_price         |avg_rating        |unique_customers|
+------------------+-----------+---------+-------------+------------------+------------------+----------------+
|Urban Chic        |Footwear   |190      |23898.0      |151.25316455696202|3.1788235294117646|106             |
|Premium Fashion   |Accessories|228      |22147.0      |121.02185792349727|2.934905660377359 |118             |
|Modern Wardrobe   |Tops       |120      |21831.0      |225.06185567010309|2.6247787610619473|85              |
|Urban Chic        |Bottoms    |148      |18276.0      |142.78125         |2.9548872180451125|97              |
|Boutique Brand    |Accessories|118      |17880.0      |173.59223300970874|3.0790476190476186|83              |
|Classic Collection|Accessories|129      |17411.0      |165.8190476190476 |3.116964285714286 |87        

### Query 7: High-Value Transaction Analysis

In [24]:
# Identify high-value transactions and patterns
df_query7 = df_sales \
    .filter(col("purchase_amount") >= 200) \
    .join(df_dim_customers, "customer_key") \
    .join(df_dim_products, "product_key") \
    .join(df_dim_payment, "payment_key") \
    .groupBy("category", "loyalty_member", "payment_method") \
    .agg(
        count("customer_key").alias("num_high_value_sales"),
        sum("purchase_amount").alias("total_high_value_revenue"),
        avg("purchase_amount").alias("avg_high_value_amount"),
        max("purchase_amount").alias("max_purchase")
    ) \
    .orderBy(desc("total_high_value_revenue")) \
    .limit(10)

print("="*80)
print("QUERY 7: HIGH-VALUE TRANSACTIONS (>$200) ANALYSIS")
print("="*80)
df_query7.show(10, truncate=False)

QUERY 7: HIGH-VALUE TRANSACTIONS (>$200) ANALYSIS


25/12/18 04:07:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:07:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 0

+-----------+--------------+--------------+--------------------+------------------------+---------------------+------------+
|category   |loyalty_member|payment_method|num_high_value_sales|total_high_value_revenue|avg_high_value_amount|max_purchase|
+-----------+--------------+--------------+--------------------+------------------------+---------------------+------------+
|Accessories|Yes           |Cash          |5                   |15758.0                 |3151.6               |4418.0      |
|Footwear   |No            |Credit Card   |4                   |15075.0                 |3768.75              |4932.0      |
|Bottoms    |Yes           |Cash          |3                   |11376.0                 |3792.0               |4872.0      |
|Tops       |Yes           |Cash          |6                   |10197.0                 |1699.5               |4859.0      |
|Accessories|Yes           |Credit Card   |4                   |10093.0                 |2523.25              |4619.0      |


### Summary: Key Business Insights

In [26]:
print("="*80)
print("ANALYTICAL SUMMARY - KEY BUSINESS INSIGHTS")
print("="*80)

# Total metrics
total_revenue = df_sales.agg(sum("purchase_amount")).collect()[0][0]
total_customers = df_sales.select("customer_key").distinct().count()
total_transactions = df_sales.count()
avg_order_value = total_revenue / total_transactions

print(f"\n📊 OVERALL METRICS:")
print(f"   Total Revenue: ${total_revenue:,.2f}")
print(f"   Total Customers: {total_customers}")
print(f"   Total Transactions: {total_transactions}")
print(f"   Average Order Value: ${avg_order_value:.2f}")

# Loyalty breakdown
loyalty_stats = df_sales \
    .join(df_dim_customers, "customer_key") \
    .groupBy("loyalty_member") \
    .agg(
        count_distinct("customer_key").alias("customers"),
        sum("purchase_amount").alias("revenue")
    ).collect()

print(f"\n🎖️  LOYALTY PROGRAM IMPACT:")
for row in loyalty_stats:
    print(f"   {row['loyalty_member']} Members: {row['customers']} customers, ${row['revenue']:,.2f} revenue")

# Category breakdown
category_stats = df_sales \
    .join(df_dim_products, "product_key") \
    .groupBy("category") \
    .agg(sum("purchase_amount").alias("revenue")) \
    .orderBy(desc("revenue")) \
    .limit(3) \
    .collect()

print(f"\n🏆 TOP 3 CATEGORIES BY REVENUE:")
for i, row in enumerate(category_stats, 1):
    print(f"   {i}. {row['category']}: ${row['revenue']:,.2f}")

print("\n" + "="*80)
print("✅ ANALYTICAL QUERIES COMPLETE!")
print("="*80)

ANALYTICAL SUMMARY - KEY BUSINESS INSIGHTS

📊 OVERALL METRICS:
   Total Revenue: $430,952.00
   Total Customers: 166
   Total Transactions: 3400
   Average Order Value: $126.75


25/12/18 04:08:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:08:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:08:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:08:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.



🎖️  LOYALTY PROGRAM IMPACT:
   No Members: 75 customers, $195,086.00 revenue
   Yes Members: 91 customers, $235,866.00 revenue

🏆 TOP 3 CATEGORIES BY REVENUE:
   1. Accessories: $102,848.00
   2. Tops: $97,113.00
   3. Bottoms: $62,541.00

✅ ANALYTICAL QUERIES COMPLETE!


25/12/18 04:08:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:08:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:08:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/12/18 04:08:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
